In [40]:
import pandas as pd
import os
import git

In [41]:
def get_git_root(path):
    git_repo = git.Repo(path, search_parent_directories=True)
    git_root = git_repo.git.rev_parse("--show-toplevel")
    return git_root

txn_date_col_name = "txn_date"
txn_description_col_name = "txn_description"
txn_amount_col_name = "txn_amount"
txn_category_col_name = "category"
txn_subcat_col_name = "sub_category"
processed_txn_filename = "categorized_txn.csv"
bank_col_name = "bank"
id_col_name = "unique_id"
txn_strftime_format = "%Y-%m-%d"
year_col_name = 'year'
month_col_name = 'month'
txn_amt_sum_col_name = 'txn_amt_sum'
num_txns_col_name = 'num_txns'

data_root_dir_append = "data"
data_raw_dir_append = "data/raw"
data_interim_dir_append = "data/interim"
data_processed_dir_append = "data/processed"

processed_txn_filename = "categorized_txn.csv"

In [42]:
git_root_path = get_git_root(os.getcwd())
interim_data_path = os.path.join(git_root_path, data_interim_dir_append)
interim_txn_path = os.path.join(interim_data_path, processed_txn_filename)

interim_txn_df = pd.read_csv(interim_txn_path, index_col= id_col_name)
interim_txn_df[txn_date_col_name] = pd.to_datetime(interim_txn_df[txn_date_col_name], format=txn_strftime_format)
#interim_txn_df[txn_date_col_name] = pd.to_datetime(interim_txn_df[txn_date_col_name], format="%m/%d/%y")

In [43]:
# interim_txn_df[txn_date_col_name] = interim_txn_df.apply(
#     lambda x: x[txn_date_col_name].strftime(txn_strftime_format),
#     axis = 1
# )

# interim_txn_df = interim_txn_df.sort_values("txn_date", ascending=False)
# interim_txn_df.to_csv(interim_txn_path)

In [44]:
interim_txn_df.tail(n=10)

,category,sub_category,txn_amount,txn_date,txn_description
unique_id,,,,,
usaa_checking_20190827_19.25,Experiences,Tickets,-19.25,2019-08-27,MAID OF THE MIST CORPO 716-2848897 NY
usaa_checking_20190826_56.29,Auto and Transport,Gas,-56.29,2019-08-26,SUNOCO 0867528201MILEPOSTLE ROY NY
usaa_checking_20190826_35.0,Vanlife,Maintenance,-35.00,2019-08-26,UNITED AUTO &amp; TRUCK SERVIMALDEN MA
usaa_checking_20190826_24.43,Auto and Transport,Ride Share,-24.43,2019-08-26,UBER TRIP HELP.UBER.COMCA
usaa_checking_20190826_13.86,Dining Out,Solo,-13.86,2019-08-26,CHIPOTLE 1580 CHICOPEE MA
usaa_checking_20190826_6.22,Groceries,Unknown,-6.22,2019-08-26,ONTARIO SERVICE #1393 LEROY NY
usaa_checking_20190826_9.0,Auto and Transport,Tolls,-9.00,2019-08-26,ITR PORTAGE TOLLS GRANGER IN
usaa_checking_20190826_9.94,Groceries,Food,-9.94,2019-08-26,DELAWARE NORTH LOGAN F&amp;BDBOSTON MA
usaa_checking_20190826_10.82,Entertainment,Spotify,-10.82,2019-08-26,Spotify USA 877-7781161 NY


In [45]:
processed_txn_df = interim_txn_df.copy()

In [46]:
processed_txn_df

,category,sub_category,txn_amount,txn_date,txn_description
unique_id,,,,,
usaa_checking_20191217_8.0,Gifts and Donations,Holiday/Birthday,-8.00,2019-12-17,TST* STIR COOKING SCHOOLTDENVER CO
usaa_checking_20191217_4.2,Dining Out,Coffee,-4.20,2019-12-17,SQ *ALLEGRO COFFEE DENVER CO
usaa_checking_20191217_21.73,Vanlife,Auto Parts,-21.73,2019-12-17,AMZN Mktp US*XA2802ID3 AMZN.COM BILLWA
usaa_checking_20191216_8.0,Education,Miscellaneous,-8.00,2019-12-16,PARCHMENT - UNIV DOCS 480-719-1646 AZ
usaa_checking_20191216_56.7,Auto and Transport,Gas,-56.70,2019-12-16,EXXONMOBIL 48225817 FRISCO CO
...,...,...,...,...,...
usaa_checking_20190826_6.22,Groceries,Unknown,-6.22,2019-08-26,ONTARIO SERVICE #1393 LEROY NY
usaa_checking_20190826_9.0,Auto and Transport,Tolls,-9.00,2019-08-26,ITR PORTAGE TOLLS GRANGER IN
usaa_checking_20190826_9.94,Groceries,Food,-9.94,2019-08-26,DELAWARE NORTH LOGAN F&amp;BDBOSTON MA


In [47]:
processed_txn_df['year'] = processed_txn_df.apply(
    lambda x: 
    x[txn_date_col_name].year
    , axis = 1
)

processed_txn_df['month'] = processed_txn_df.apply(
    lambda x: 
    x[txn_date_col_name].month
    , axis = 1
)

processed_txn_df['day'] = processed_txn_df.apply(
    lambda x: 
    x[txn_date_col_name].day
    , axis = 1
)

processed_txn_df['weekday'] = processed_txn_df.apply(
    lambda x: 
    x[txn_date_col_name].strftime("%A")
    , axis = 1
)

# get expenses to be positive
processed_txn_df[txn_amount_col_name] = processed_txn_df.apply(
    lambda x: 
    -1 * x[txn_amount_col_name] if x[txn_category_col_name] != 'Salary' else x[txn_amount_col_name]
    , axis = 1
)

In [48]:
processed_txn_df

,category,sub_category,txn_amount,txn_date,txn_description,year,month,day,weekday
unique_id,,,,,,,,,
usaa_checking_20191217_8.0,Gifts and Donations,Holiday/Birthday,8.00,2019-12-17,TST* STIR COOKING SCHOOLTDENVER CO,2019,12,17,Tuesday
usaa_checking_20191217_4.2,Dining Out,Coffee,4.20,2019-12-17,SQ *ALLEGRO COFFEE DENVER CO,2019,12,17,Tuesday
usaa_checking_20191217_21.73,Vanlife,Auto Parts,21.73,2019-12-17,AMZN Mktp US*XA2802ID3 AMZN.COM BILLWA,2019,12,17,Tuesday
usaa_checking_20191216_8.0,Education,Miscellaneous,8.00,2019-12-16,PARCHMENT - UNIV DOCS 480-719-1646 AZ,2019,12,16,Monday
usaa_checking_20191216_56.7,Auto and Transport,Gas,56.70,2019-12-16,EXXONMOBIL 48225817 FRISCO CO,2019,12,16,Monday
...,...,...,...,...,...,...,...,...,...
usaa_checking_20190826_6.22,Groceries,Unknown,6.22,2019-08-26,ONTARIO SERVICE #1393 LEROY NY,2019,8,26,Monday
usaa_checking_20190826_9.0,Auto and Transport,Tolls,9.00,2019-08-26,ITR PORTAGE TOLLS GRANGER IN,2019,8,26,Monday
usaa_checking_20190826_9.94,Groceries,Food,9.94,2019-08-26,DELAWARE NORTH LOGAN F&amp;BDBOSTON MA,2019,8,26,Monday


In [49]:
# cat_detail_df = processed_txn_df.groupby([txn_category_col_name, 'year', 'month']).agg(
#     sum_txns = pd.NamedAgg(column = txn_amount_col_name, aggfunc=sum),
#     num_txns = pd.NamedAgg(column = txn_amount_col_name, aggfunc=pd.Series.nunique)
# ).reset_index()

subcat_detail_df = processed_txn_df.groupby([txn_category_col_name, txn_subcat_col_name, 'year', 'month']).agg(
    txn_amt_sum = pd.NamedAgg(column = txn_amount_col_name, aggfunc=sum),
    num_txns = pd.NamedAgg(column = txn_amount_col_name, aggfunc=pd.Series.nunique)
).reset_index()

full_cat_detail_df = subcat_detail_df.groupby([txn_category_col_name, year_col_name, month_col_name]).agg(
    txn_amt_sum = pd.NamedAgg(column = txn_amt_sum_col_name, aggfunc=sum),
    num_txns = pd.NamedAgg(column = num_txns_col_name, aggfunc=sum),
).reset_index()

cat_detail_df_expenses = subcat_detail_df[~subcat_detail_df[txn_subcat_col_name]\
    .isin(['Credit Card Payment'])].reset_index()

cat_detail_df_expenses = cat_detail_df_expenses[~cat_detail_df_expenses[txn_category_col_name]\
    .isin(['Investments', 'Income'])]\
    .groupby([txn_category_col_name, year_col_name, month_col_name])\
    .agg(
        txn_amt_sum = pd.NamedAgg(column = txn_amt_sum_col_name, aggfunc=sum),
        num_txns = pd.NamedAgg(column = num_txns_col_name, aggfunc=pd.Series.nunique)
    ).reset_index()

subcat_detail_df['avg_per_txn'] = subcat_detail_df[txn_amt_sum_col_name] / subcat_detail_df[num_txns_col_name]
cat_detail_df_expenses['avg_per_txn'] = cat_detail_df_expenses[txn_amt_sum_col_name] / cat_detail_df_expenses[num_txns_col_name]
full_cat_detail_df['avg_per_txn'] = full_cat_detail_df[txn_amt_sum_col_name] / full_cat_detail_df[num_txns_col_name]

In [50]:
col_name_to_fill = txn_category_col_name
df = cat_detail_df_expenses.copy()
df_columns = df.columns
years = df[year_col_name].unique()

for year in years:
    
    year_df = df[df[year_col_name] == year]
    column_val_array = year_df[col_name_to_fill].unique()
    
    for col_val in column_val_array:
        
        col_val_df = year_df[year_df[col_name_to_fill] == col_val]
        months_array = year_df[month_col_name].unique()
        
        for month in months_array:
            
            df_subset = col_val_df[(col_val_df[year_col_name] == year) & (col_val_df[month_col_name] == month)] 
            row_exists = True if len(df_subset) > 0 else False
            
            if row_exists is False:
                new_row = {
                    year_col_name : year,
                    month_col_name : month,
                    col_name_to_fill : col_val
                }
                
                new_row_full = {
                    col : (0 if col not in new_row.keys() else new_row[col])
                    for col in df_columns
                }
                
                df = df.append(new_row_full, ignore_index=True)
#                 (some_key if condition else default_key):(something_if_true if condition 
#           else something_if_false) for key, value in dict_.items() }
                    
                #new_row = map(lambda x: new_row[x] = 0 if x not in new_row.keys() else new_row[x] = new_row[x], df_columns) 
                print("-- {} / {} / {}".format(col_val, year, month))

-- Bills and Utilities / 2019 / 8
-- Bills and Utilities / 2019 / 10
-- Bills and Utilities / 2019 / 11
-- Bills and Utilities / 2019 / 12
-- Education / 2019 / 8
-- Education / 2019 / 9
-- Education / 2019 / 10
-- Entertainment / 2019 / 12
-- Experiences / 2019 / 9
-- Experiences / 2019 / 12
-- Fees and Charges / 2019 / 8
-- Fees and Charges / 2019 / 12
-- Food and Dining / 2019 / 8
-- Food and Dining / 2019 / 9
-- Food and Dining / 2019 / 12
-- Gifts and Donations / 2019 / 8
-- Health and Fitness / 2019 / 8
-- Health and Fitness / 2019 / 9
-- Health and Fitness / 2019 / 12
-- Hobbies / 2019 / 8
-- Hobbies / 2019 / 9
-- Hobbies / 2019 / 10
-- Hobbies / 2019 / 11
-- Miscellaneous / 2019 / 8
-- Miscellaneous / 2019 / 10
-- Personal Care / 2019 / 8
-- Shopping / 2019 / 8
-- Uncategorized / 2019 / 12


In [51]:
df.sort_values([txn_category_col_name, year_col_name, month_col_name]).to_csv('./filled_df.csv', index=False)

In [29]:
new_row

{'year': 2019, 'month': 12, 'category': 'Uncategorized'}

In [16]:
col_name_to_fill = txn_category_col_name
df = cat_detail_df_expenses.copy()
years = df[year_col_name].unique()

for year in years:
    year_df = df[df[year_col_name] == year]
    cleaned_df = col_val_subset(year_df, col_name_to_fill)
    
def col_val_subset(year_df, col_name):
    
    column_val_array = year_df[col_name_to_fill].unique()
    
    for col_val in column_val_array:
        cleaned_df = months_subset(year_df, col_name, col_val)

    return cleaned_df 
        
def months_subset(year_df, col_name, col_val):
    
    months_array = year_df[month_col_name].unique()

array([2019])

In [17]:
cat_detail_df_expenses

,category,year,month,txn_amt_sum,num_txns,avg_per_txn
0,Auto and Transport,2019,8,167.83,2.0,83.915000
1,Auto and Transport,2019,9,253.95,4.0,63.487500
2,Auto and Transport,2019,10,427.92,3.0,142.640000
3,Auto and Transport,2019,11,337.97,3.0,112.656667
4,Auto and Transport,2019,12,194.96,1.0,194.960000
5,Bills and Utilities,2019,9,500.00,1.0,500.000000
6,Dining Out,2019,8,13.86,1.0,13.860000
7,Dining Out,2019,9,847.56,4.0,211.890000
8,Dining Out,2019,10,254.95,4.0,63.737500
9,Dining Out,2019,11,400.13,4.0,100.032500


In [ ]:
def add_missing_cats(df, col_name):
    
    


In [ ]:
cat_detail_df_wo_cc.to_csv('./cat_detail_df_wo_cc.csv', index=False)

In [ ]:
cat_detail_df.sort_values('sum_txns')

In [ ]:
subcat_detail_df[(subcat_detail_df[txn_category_col_name] == 'Auto and Transport') 
                 & (subcat_detail_df['month'] == 10)]

In [ ]:
cat_specific_df.pivot(index='month', columns='sub_category', values='sum_txns').plot(kind='bar')